Read the rows from the Viasat dataset, insert a row in a db table for every trip (start to stop, individuated by distance travelled)

In [16]:
import pymysql, pymysql.cursors, csv, more_itertools

In [17]:
db = None

def get_db():
    try:
        db.close()
    except Exception as e:
        #out(e)
        pass

    db = pymysql.connect(host="192.168.56.1",
                         user="paul",
                         passwd="paul21",
                         db="bigdive")
    return db

In [18]:
def out(*mex):
    if(False):
        for m in mex:
            out(m)

In [19]:
nr=0
def insert_trip(db, data):
    sql = """INSERT INTO `trips` 
    (car_id, start_pos, end_pos, start_time, end_time, length, points, parking)
    VALUES (%s, ST_PointFromText(%s), ST_PointFromText(%s), %s, %s, %s, ST_LineFromText(%s), %s)"""
    
    out(sql%data)
    
    global nr
    with db.cursor() as cursor:
        cursor.execute(sql, data)
        nr+=1
        if nr > 1000:
            db.commit()
            nr=0

In [20]:
fieldnames = ['id', 'datetime', 'lat', 'long', 'speed', 'heading', 'gps_quality', 'event_type', 'abc', 'meters']

def process_csv(db, file):
    with open(file, newline='') as f:
        reader = more_itertools.peekable(csv.DictReader(f, fieldnames=fieldnames))
        points = []
        start_row = None
        end_row = None
        oldlen = 0
        curr_carid = 0
        n = 0
        prev_trip = None
        
        def process_points(points):
            if len(points) == 1:
                points.append(points[0])
                
            # processa i punti accumulati per il viaggio
            out("\nTrip car:", curr_carid, points, "\n")
            # car_id, start_pos, end_pos, start_time, end_time, length, points
            start_point = "POINT(%f %f)" % (float(start_row['long']), float(start_row['lat']))
            end_point = "POINT(%f %f)" % (float(end_row['long']), float(end_row['lat']))
            line = "LINESTRING(%s)" % ", ".join(["%f %f" % (float(a['long']), float(a['lat'])) for a in points])
            insert_trip(db, (int(curr_carid), start_point, end_point, start_row['datetime'], end_row['datetime'], oldlen, line, 0))
        
        def process_stop(prev_trip):
             # processa i punti accumulati per il viaggio
            out("\nStop car:", curr_carid, "\n")
            # car_id, start_pos, end_pos, start_time, end_time, length, points
            start_point = "POINT(%f %f)" % (float(prev_trip['long']), float(prev_trip['lat']))
            end_point = "POINT(%f %f)" % (float(start_row['long']), float(start_row['lat']))
            line = "LINESTRING(%f %f, %f %f)" % (float(prev_trip['long']), float(prev_trip['lat']), float(start_row['long']), float(start_row['lat']))
            out(prev_trip)
            insert_trip(db, (int(curr_carid), start_point, end_point, prev_trip['datetime'], start_row['datetime'], 0, line, 1))
        
        def process_csv_row(row):
            points.append({'lat': row['lat'], 'long': row['long']})
        
        for row in reader:
            if start_row is None:
                start_row = row
                
            curr_carid = row['id']
            n+=1
            out(row)
            process_csv_row(row)
            
            nomorerows = 0
            
            try:
                reader.peek()
            except StopIteration:
                # no more rows
                nomorerows = 1
            
            if nomorerows or reader.peek()['id'] != row['id'] :
                end_row = row
                if prev_trip is not None:
                    process_stop(prev_trip)
                process_points(points)
                points = []
                oldlen = 0
                start_row = None
                prev_trip = None
                out("Nuova car")
            elif int(row['meters']) > int(reader.peek()['meters']):
                end_row = row
                if prev_trip is not None:
                    process_stop(prev_trip)
                process_points(points)
                points = []
                oldlen = 0
                start_row = None
                prev_trip = {
                    'lat' : row['lat'],
                    'long' : row['long'],
                    'datetime' : row['datetime']
                }
            #if n == 100:
            #    break
                
            oldlen = int(row['meters'])

In [21]:
def use_db():
    db = pymysql.connect(host="192.168.56.1",
                         user="paul",
                         passwd="paul21",
                         db="bigdive")

    try:
        with db.cursor() as cursor:
            sql = "SELECT car_id, numpoints FROM cars limit 10"
            data = []
            cursor.execute(sql, data)
            result = cursor.fetchone()
            out(result)
    finally:
        db.close()

In [22]:
try:
    db = get_db()
    process_csv(db, 'vem_dataset_filename.csv')
finally:
    db.commit()
    db.close()
    print("FINISH")

FINISH
